In [1]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Seed for reproducibility
torch.manual_seed(0)

In [4]:
# Synthetic Data Generation
def generate_synthetic_data(samples=1000, features=10, sequence_length=50):
    """
    Generate synthetic EMG data and corresponding arm movement sequences.
    :param samples: Number of data samples
    :param features: Number of features in EMG data
    :param sequence_length: Length of the output sequence
    :return: emg_data, arm_movements
    """
    emg_data = torch.randn(samples, sequence_length, features)
    arm_movements = torch.randn(samples, sequence_length, features)
    return emg_data, arm_movements

# Generate synthetic data
emg_data, arm_movements = generate_synthetic_data()

In [5]:
# Create DataLoader
batch_size = 32
dataset = TensorDataset(emg_data, arm_movements)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [6]:
# Simple RNN Model
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x


In [7]:
# Complicated RNN Model (e.g., multi-layer)
class ComplicatedRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2):
        super(ComplicatedRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x

In [8]:
# Initialize models
input_size = emg_data.shape[2]
hidden_size = 64  # Example size
output_size = arm_movements.shape[2]
simple_model = SimpleRNN(input_size, hidden_size, output_size)
complicated_model = ComplicatedRNN(input_size, hidden_size, output_size)

In [9]:
# Training parameters
epochs = 5
learning_rate = 0.001

# Loss and optimizer
criterion = nn.MSELoss()
optimizer_simple = optim.Adam(simple_model.parameters(), lr=learning_rate)
optimizer_complicated = optim.Adam(complicated_model.parameters(), lr=learning_rate)

In [10]:
# Training function
def train_model(model, optimizer):
    model.train()
    for epoch in range(epochs):
        for i, (inputs, targets) in enumerate(dataloader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            if i % 100 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

In [11]:
# Train models
train_model(simple_model, optimizer_simple)
train_model(complicated_model, optimizer_complicated)

Epoch [1/5], Step [1/32], Loss: 1.0220
Epoch [2/5], Step [1/32], Loss: 0.9940
Epoch [3/5], Step [1/32], Loss: 0.9779
Epoch [4/5], Step [1/32], Loss: 1.0011
Epoch [5/5], Step [1/32], Loss: 1.0057
Epoch [1/5], Step [1/32], Loss: 1.0264
Epoch [2/5], Step [1/32], Loss: 0.9894
Epoch [3/5], Step [1/32], Loss: 0.9824
Epoch [4/5], Step [1/32], Loss: 0.9897
Epoch [5/5], Step [1/32], Loss: 1.0132


In [12]:
# Save checkpoints
torch.save(simple_model.state_dict(), 'simple_model_checkpoint.pth')
torch.save(complicated_model.state_dict(), 'complicated_model_checkpoint.pth')